In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0041277408599853516
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 17.839390484526664
200000 31.091184778091908
300000 32.78205014989029
400000 32.49205493547613
500000 26.513252460014005
600000 31.313337132249554
700000 31.46413258700872
800000 29.97885520463494
900000 23.90916943830215
1000000 13.08138411467716
1100000 28.136777235116096
1200000 24.871017681767146
1300000 26.55096035085293
1400000 25.62357381068275
1500000 27.595979790648002
1600000 24.89650368306197
1700000 27.378218956396807
1800000 25.608812526928215
1900000 10.222760364188076
2000000 28.16753011704152
2100000 30.92113640518067
2200000 32.21148959812924
2300000 30.706753215576104
2400000 32.90165904721784
2500000 29.09292666090534
2600000 31.051063893178558
2700000 29.297134284166045
2800000 19.572774784200163
2900000 22.66389360715031
3000000 30.6

29300000 23.333320842147003
29400000 24.73021793297506
29500000 24.1712191139599
29600000 12.180646370965757
29700000 24.08821976863809
29800000 24.4855477745755
29900000 24.775836530847723
30000000 22.114386539059723
30100000 13.308345329066464
30200000 24.537023746097752
30300000 23.651028231657335
30400000 23.11512951823643
30500000 22.399755893892277
30600000 20.98825952962647
30700000 15.986410586680918
30800000 14.91321562772037
30900000 21.375164341376795
31000000 22.011259987993668
31100000 22.785511513111675
31200000 23.146518252334374
31300000 24.50459200929163
31400000 15.1865872296436
31500000 20.815354898968916
31600000 23.643030477232127
31700000 25.020881198355706
31800000 24.191326017595678
31900000 17.953107727322273
32000000 18.635110009022252
32100000 25.124177703554675
32200000 23.401669744497845
32300000 24.07339537174062
32400000 22.201679684496572
32500000 21.275568928921345
32600000 8.878649578459124
32700000 22.361062773455693
32800000 21.256922444260848
329000

59100000 11.287002843234164
59200000 20.268992324019997
59300000 20.607677667091725
59400000 21.20317119540133
59500000 21.756363803133844
59600000 22.342479397135065
59700000 21.603188922752484
59800000 21.482287813986012
59900000 20.85413322681699
60000000 21.510400992404698
60100000 9.34490890903003
60200000 19.098724877178558
60300000 18.929889027051214
60400000 19.374026617197753
60500000 9.946279417957742
60600000 17.605800116251583
60700000 19.23300979891801
60800000 18.718963556346495
60900000 16.903094716310267
61000000 13.640963718328138
61100000 20.204021953945617
61200000 20.993820843589763
61300000 20.670932476416528
61400000 21.1321249854962
61500000 21.91247888748791
61600000 21.72884026352751
61700000 21.8000725196546
61800000 20.431909144981926
61900000 17.374582316181698
62000000 12.414707001420844
62100000 19.182939107413237
62200000 19.830862323857843
62300000 19.602681597834355
62400000 7.681244808116744
62500000 19.40280278403206
62600000 18.733874272507148
627000

88900000 17.589087028024615
89000000 17.518659844290962
89100000 17.738976433545485
89200000 18.52291588284886
89300000 19.218106133948577
89400000 19.35348311438759
89500000 19.185658087457725
89600000 18.067400331313287
89700000 17.310895119597692
89800000 17.286629201819313
89900000 17.85447573441595
90000000 16.92474043831533
90100000 8.347848319819063
90200000 16.70437216691119
90300000 6.752979389538501
90400000 17.590619343747527
90500000 8.808281515793542
90600000 17.518441082607243
90700000 17.54052737601177
90800000 17.090344756570744
90900000 17.484033786249338
91000000 17.567044168140917
91100000 18.740711170024678
91200000 19.19417386049076
91300000 19.811269800065606
91400000 18.56398336658343
91500000 17.094594086923845
91600000 17.475180717696897
91700000 17.64269626878012
91800000 17.452908068232546
91900000 16.749824460938335
92000000 8.74718051400424
92100000 13.79119850041535
92200000 10.709981515907696
92300000 11.90164982680841
92400000 13.90425554154779
92500000 

118000000 1.7438208819256427
118100000 1.7428571402344373
118200000 1.7419109643933608
118300000 1.7386146308765211
118400000 1.7405765550838577
118500000 1.7413240469129483
118600000 1.7403619134312571
118700000 1.7311150836686888
118800000 1.7417768896506163
118900000 1.7553417555555104
119000000 1.7493015438102153
119100000 2.579675330697318
119200000 23.10349177154805
119300000 21.839730977314044
119400000 21.40721282544912
119500000 21.04648956718332
119600000 20.670704920986214
119700000 20.487960971824076
119800000 20.099276378672275
119900000 20.745730727695495
120000000 10.814252662593443
120100000 2.7128291631258135
120200000 18.989439011657062
120300000 19.98662886536818
120400000 20.759993171025634
120500000 20.156735701504875
120600000 21.636138126587245
120700000 20.789972239917216
120800000 21.802554568020824
120900000 22.10801837905283
121000000 21.41842287095629
121100000 17.93076686891031
121200000 17.499788303138075
121300000 16.578540970943532
121400000 17.163339361

...

233000000 2.109854076796237
233100000 2.2424542128016647
233200000 2.2773205584872365
233300000 2.4045935873253512
233400000 2.1780086576998006
233500000 1.0131639607162972
233600000 0.5462292250445552
233700000 1.61579983451881
233800000 2.1618099504923123
233900000 2.555851589440518
234000000 2.5805236973002916
234100000 2.4135794098163545
234200000 2.507824036563277
234300000 2.5377070436210034
234400000 2.176896347861707
234500000 2.6230566557914745
234600000 3.0385507732937964
234700000 2.0995162740266022
234800000 2.4974397601546214
234900000 2.656621979206456
235000000 3.1196939947260915
235100000 3.6496111636021533
235200000 4.108099966672794
235300000 4.18982700287497
235400000 1.0245223585313359
235500000 1.0153746965028279
235600000 4.579398060656716
235700000 4.475248189586087
235800000 4.47627201930065
235900000 4.505508753968359
236000000 4.425519309953292
236100000 4.39526209654144
236200000 4.0907307712726
236300000 4.580087576052751
236400000 5.007822032172336
23650000

261900000 3.151527059056862
262000000 3.083488366547495
262100000 3.1201854894578727
262200000 3.046116795374023
262300000 3.1480454938799043
262400000 3.726697706303342
262500000 3.8818937605686847
262600000 3.4086466086280938
262700000 3.1258590374413924
262800000 3.1457635670367114
262900000 3.3030839855623544
263000000 3.2825028729153605
263100000 3.053878542161266
263200000 3.501019551258167
263300000 0.7322917232361303
263400000 0.7242983092379173
263500000 2.7784897730454903
263600000 3.2623836933774384
263700000 3.217011466442999
263800000 3.4613595695948303
263900000 3.2181685809240337
264000000 3.006265162917617
264100000 3.332133576538334
264200000 3.215534038654533
264300000 4.4698239462338485
264400000 4.072140190246403
264500000 3.56839686750974
264600000 3.388704352600349
264700000 3.702752715002835
264800000 3.785890716628592
264900000 3.4950549016812507
265000000 3.1358745004466275
265100000 3.1681295760120523
265200000 0.7212528197453882
265300000 1.3036577836091419
2

290700000 1.930606579457197
290800000 2.2071210414067806
290900000 2.1043979884474178
291000000 2.051484675244434
291100000 2.1359164006049536
291200000 0.8064989126653141
291300000 0.5030951588600411
291400000 1.809805467321555
291500000 1.9399277887834507
291600000 2.0428779017858107
291700000 2.146027051688999
291800000 2.1980428609461717
291900000 1.9562499980890817
292000000 2.1425481380079487
292100000 2.273691742265314
292200000 2.590829663219972
292300000 2.1292173258197504
292400000 2.2215880656050677
292500000 1.8949600772839021
292600000 2.0506332970060703
292700000 2.1279623091449706
292800000 2.2200075825372108
292900000 1.9988348380065957
293000000 1.7811136005239652
293100000 0.49285073329578505
293200000 0.5045505619574387
293300000 2.4436484561007257
293400000 1.968460033293489
293500000 2.1586092983432112
293600000 2.099461960340513
293700000 2.1782650458410777
293800000 1.9426214610667127
293900000 2.2441760762703518
294000000 2.218964118911346
294100000 2.4422301826

318500000 1.2130984176524215
318600000 1.3052915197455583
318700000 1.2799841575359105
318800000 1.3016230233942303
318900000 1.1728176318576666
319000000 1.2295830009629132
319100000 1.1287603216995563
319200000 1.276805611559328
319300000 1.2763479126369257
319400000 1.3047889787280988
319500000 1.1935344661919043
319600000 1.1614225952759385
319700000 1.1812744298546314
319800000 1.4333807325557935
319900000 1.2070241041166883
320000000 1.2089725204368333
320100000 1.1309316826841036
320200000 1.110386614939332
320300000 1.236516304610398
320400000 1.1872569646716018
320500000 1.2169666721405983
320600000 1.2720577710375751
320700000 1.1226963293098111
320800000 1.123722201041003
320900000 1.1295918836915726
321000000 1.155271084598253
321100000 1.221425929134945
321200000 1.2890023394625154
321300000 1.1030667718274196
321400000 1.0350778094798028
321500000 1.034342467382239
321600000 1.1112333675239716
321700000 1.1057777046565513
321800000 1.1378533674386873
321900000 1.108107130

345400000 0.09220711600198349
345500000 0.20482357326686382
345600000 0.13766443655507435
345700000 0.17932670237818357
345800000 0.1528879451789293
345900000 0.09842671713972423
346000000 0.08497445998314024
346100000 0.07272282571062445
346200000 0.09054552904778057
346300000 0.07615299400664369
346400000 0.12260520253182451
346500000 0.11748637816657954
346600000 0.1384443103118837
346700000 0.0807703472237554
346800000 0.11722517318105698
346900000 0.10519779173624516
347000000 0.06515668773124283
347100000 0.0789415757317245
347200000 0.11160728814014792
347300000 0.08883413017681571
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.058803728571110304
347500000 0.011312636693447828
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
cases of this type: 369
best s